# ARTIST CENTRALITY _part I_
## Learning from Networks project 2022-2023

# TODO: add info of our group

# TODO: add short descritpion of the project here

Let's start by importing libraries that we'll use

In [1]:
import networkx as nx 
import time
import random as rnd
import math

Let's build our graph starting with adding the nodes

In the file *nodes.csv* datas is rappresented in the following format: ___"spotify_id,name,followers,popularity,genres,chart_hits"___.

We are interested in the spotify ID, in the name, in the number of followers and in the popularity index.

In the dataset there are some duplicates so we need to check we are not adding any by mistake and that the one we added are the one with the information we are interested (i.e. a duplicate can have the number of followers setted to 0)

N.B.: we need to be carefull splitting the line based on commas since an artist name can have a comma aswell
- without comma: 48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']

- with comma: 7c1HgFDe8ogy5NOZ1ANCJQ,"Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"

In [2]:
G = nx.Graph()

#Load dataset with nodes infos
f = open('dataset/nodes.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

discarded_counter=0

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #First we extrapolate the id
    current_id, tmp = line.split(',', 1)
    
    #initialize other variables
    current_artist = current_followers = current_popularity = ''
    
    #Now we divide the 2 cases, artist with comma in their name and artists without,
    #we do that by checking if the first character is equals to '"'
    if tmp[0] != '"':
        #i.e. tmp = Byklubben,1738.0,24,"['nordic house', 'russelater']",['no (3)']
        current_artist, current_followers, current_popularity, tmp = tmp.split(',', 3)
    else:
        #i.e. tmp = "Car, the garden",110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_artist, tmp = tmp.split('"', 2)
        #i.e. tmp = ,110672.0,51,"['k-indie', 'korean pop']","['id (1)', 'my (1)', 'th (1)']"
        empty, current_followers, current_popularity, tmp = tmp.split(',', 3)

    #Now let's try converting followers and popularity index currently string to ints,
    #in case we can't we skip the current node reporting a message
    try:
        current_followers = int(float(current_followers))
        current_popularity = int(current_popularity)                
    except ValueError as ve:
        print('ValueError occured while converting string to int. Node ID:', current_id)
        discarded_counter += 1
        continue
    
    #if the artist is relevant enough then we add him to the graph
    if current_popularity >= 40:
        #if the ID is already a node we check we have the correct informations
        if G.has_node(current_id):
            #if the number of followers of the current line is greater than the numbers of followers already addeed
            #to the graph we just update the corresponding label
            if G.nodes[current_id]['followers'] < current_followers:
                G.nodes[current_id]['followers'] = current_followers
            #same with popularity
            if G.nodes[current_id]['popularity'] < current_popularity:
                G.nodes[current_id]['popularity'] = current_popularity
        
        #else we add it to the graph
        else:
            #add new node with mapped int as key and artist and followers as lables
            G.add_node(current_id, artist=current_artist, followers=current_followers, popularity=current_popularity, path_sum=0)
            
# Close opend file
f.close()
            
#print results
print(discarded_counter,"nodes have been discarded because of bad formatting!")
print(G.number_of_nodes(),"nodes have been added successfully")

ValueError occured while converting string to int. Node ID: 4Jgl9FmNQF6ontIRyY19Ig
ValueError occured while converting string to int. Node ID: 3cCFieWefBXyyDRsjNuArE
ValueError occured while converting string to int. Node ID: 1lLHQcDQFM03FcxZ5mQimA
ValueError occured while converting string to int. Node ID: 7ti7Mdu4BTfKOYWcI1Q6h8
ValueError occured while converting string to int. Node ID: 7estJE1m5cJnQs3Rc4iar0
5 nodes have been discarded because of bad formatting!
28621 nodes have been added successfully


Now let's add the edges

In the file *edges.csv* datas is rappresented in the following format: ___"id_0,id_1"___.

We need to check the validity of the edge before adding it because some IDs are not on the *nodes.csv* dataset

In [3]:
#Load dataset with edges info
f = open('dataset/edges.csv', "r", encoding="utf8")

# skip the first line in the input file since it contains dataset description
f.readline()

while True:
    line = f.readline().strip()
    
    #empty line = EOF
    if line == '':
        break
        
    #extrapolate id_0 and id_1 from the current line
    id_0, id_1 = line.split(',', 1)
    
    #if the indices are both valid then we add the edge
    if G.has_node(id_0) and G.has_node(id_1):
        G.add_edge(id_0, id_1)
        
# Close opend file
f.close()

print(G.number_of_edges(),"edges have been added successfully")

118141 edges have been added successfully


Since the graph consist in a big main connected component and some really small (in comparisson) connected components/isolated nodes we take in consideration only the main connected component.

To do so we clean up our graph removing those elements disconnected to the main component

In [4]:
print(G.number_of_nodes(),"nodes before clean up")
print(G.number_of_edges(),"edges before clean up")

nodes_main_component = max(nx.connected_components(G), key=len)
G = G.subgraph(nodes_main_component)

print(G.number_of_nodes(),"nodes after clean up")
print(G.number_of_edges(),"edges after clean up")

28621 nodes before clean up
118141 edges before clean up
26389 nodes after clean up
117766 edges after clean up


Let's run the exact algorithm for closeness centrality and let's also calculate the time of computation

In [5]:
start_time = time.time()
exact_closeness_centrality = nx.closeness_centrality(G)
end_time = time.time()

#let's sort the results based on the valu
exact_closeness_centrality = {k: v for k, v in sorted(exact_closeness_centrality.items(),reverse=True , key=lambda item: item[1])}

#let's print the results
#print(exact_closeness_centrality)
print("the exact closeness centralities have been computed in %s seconds" %(end_time - start_time))

#Now let's store the results
f = open('results/results_exact_closeness_centrality.txt', "w", encoding="utf8")
for key, value in exact_closeness_centrality.items():
    f.write('%s:%s\n' %(key, value))

# Close opend file
f.close()

KeyboardInterrupt: 

Let's now define the function for computing the approximated closeness centrality based on Eppstein-Wang Algorithm

In [30]:
def ApproximateClosenessCentrality(G, k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0
        
    for i in range(k):
        #pick one node uniformally at random
        random_node = rnd.choice(list(G.nodes()))
        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities      

In [31]:
def CCK_ApproximateClosenessCentrality(G,k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0

    sample_S0 = []
    all_Ws = []
    all_pv = []
    size_S0 = 10

    for i in range(size_S0):
        sample_S0.append(rnd.choice(list(G.nodes())))
    for s in sample_S0 :
        actualWs = 0
        sssp = nx.shortest_path_length(G, source=s)
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
        for n in G :
            actualWs = actualWs + G.nodes[n]['path_sum']
        all_Ws.append(actualWs)
    
    for n1 in G :
        pv = 1/len(list(G.nodes()))
        max = 0
        for s in sample_S0:
            sssp = nx.shortest_path_length(G, source=s)
            for n, path_lenght in sssp.items():
                G.nodes[n]['path_sum'] += path_lenght
            if max < G.nodes[n1]['path_sum'] :
                max = G.nodes[n1]['path_sum']
        all_pv.append(pv)

    for i in range(k):
        #pick one node based on pv (Poisson sampling)
        random_node = rnd.choices(list(G.nodes()), all_pv, 1)
        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities    

Now let's define a function for calculating the approximation using a weighted probability(instead of uniform probability) based on the degree of each node

In [32]:
def ApproximateClosenessCentrality_NodeDegree(G, k):
    #make sure lable sum is equals to 0 for every node
    for n in G:
        G.nodes[n]['path_sum'] = 0
        
    #make a list of degree of nodes. The second elements of the list "G.degree" shows the degree of each node
    nodes_degree = [x[1] for x in G.degree]
    
    for i in range(k):
        #pick one node at random with weights based on the degree of the nodes 
        random_node = rnd.choices(list(G.nodes()), weights=list(nodes_degree), k=1)[0]

        #solve sssp with picked node as source
        sssp = nx.shortest_path_length(G, source=random_node)
        #update partial sum of distancies for each node
        for n, path_lenght in sssp.items():
            G.nodes[n]['path_sum'] += path_lenght
    centralities = {}
    #compute final approximation of centrality for each node
    for n in G:
        if G.nodes[n]['path_sum'] == 0:
            centralities[n] = 0
        else:
            centralities[n] = 1/((G.number_of_nodes()*G.nodes[n]['path_sum'])/(k*(G.number_of_nodes()-1)))
    #return dicionary containg pairs (node, approximatedcentrality)
    return centralities 

Now let's run the approximated algorithm for closeness centrality with epsilon = 0.1 and let's also calculate the time of computation

In [ ]:
#let's compute the value k of the number of iteration we have to do with epsilon = 0.1
k = int(math.log(G.number_of_nodes(),2)/0.01)

start_time = time.time()
approximated_closeness_centrality_1 = ApproximateClosenessCentrality(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_1 = {k: v for k, v in sorted(approximated_closeness_centrality_1.items(),reverse=True , key=lambda item: item[1])}


#let's print the results
#print(approximated_closeness_centrality_1)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_epsilon_0_1.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_1.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opend file
f.close()

Now let's compute some statistics like:
- mean exact closeness centrality
- mean approximated closeness centrality
- mean error
- standard deviations
- top 10 artist using exact closeness centrality
- top 10 artist using approximated closeness centrality

In [ ]:
mean_exact_centralities = 0
mean_approximated_centralities_1 = 0
mean_errors_1 = 0
std_dev_exact_centralities = 0
std_dev_approximated_centralities_1 = 0
std_dev_errors_1 = 0
G_size = G.number_of_nodes()

#First we compute the means
#let's start by summing
for n in G:
    mean_exact_centralities += exact_closeness_centrality[n]
    mean_approximated_centralities_1 += approximated_closeness_centrality_1[n]
mean_errors_1 = mean_approximated_centralities_1 - mean_exact_centralities

#now let's divide by the number of nodes to compute the final values
mean_exact_centralities = mean_exact_centralities/G_size
mean_approximated_centralities_1 = mean_approximated_centralities_1/G_size
mean_errors_1 = mean_errors_1/G_size

#Now we compute the standard deviations
#Let's start by computing the summation
for n in G:
    tmp = exact_closeness_centrality[n] - mean_exact_centralities
    std_dev_exact_centralities += tmp*tmp
    tmp = approximated_closeness_centrality_1[n] - mean_approximated_centralities_1
    std_dev_approximated_centralities_1 += tmp*tmp
    tmp = approximated_closeness_centrality_1[n] - exact_closeness_centrality[n] - mean_errors_1
    std_dev_errors_1 += tmp*tmp

#Let's now compute the final value dividing by the size and computing the square root of the variance
std_dev_exact_centralities = (std_dev_exact_centralities/G_size)**0.5
std_dev_approximated_centralities_1 = (std_dev_approximated_centralities_1/G_size)**0.5
std_dev_errors_1 = (std_dev_errors_1/G_size)**0.5

#Let's print the results regarding the exact centralities
print("The mean of the exact centralities is:", mean_exact_centralities)
print("The standard deviation of the exact centralities is:", std_dev_exact_centralities)

#Let's print the top 10 computed with exact algorithm
position=1
for key, value in exact_closeness_centrality.items():
    key_node = G.nodes()[key]
    print(f'position number {position}:')
    print(f"   artist: {key_node['artist']}\n   followers: {key_node['followers']}\n   popularity: {key_node['popularity']}\n   centrality: {value}")
    position +=1
    if position > 10:
        break


#Let's print the results regarding the approximated centralities and their errors
print("Results of the approximation algorithm with epsilon = 0.1")
print("The mean of the approximated centralities is:", mean_approximated_centralities_1)
print("The standard deviation of the approximated centralities is:", std_dev_approximated_centralities_1)
print("The mean of the errors is:", mean_errors_1)
print("The standard deviation of the errors is:", std_dev_errors_1)

#Let's print the top 10 computed with approximated algorithm
position=1
for key, value in approximated_closeness_centrality_1.items():
    key_node = G.nodes()[key]
    print(f'position number {position}:')
    print(f"   artist: {key_node['artist']}\n   followers: {key_node['followers']}\n   popularity: {key_node['popularity']}\n   centrality: {value}")
    position +=1
    if position > 10:
        break

#Let's store the results
stats_file = open('results/statistics.txt', "w", encoding="utf8")
stats_file.write(f"The mean of the exact centralities is: {mean_exact_centralities}\n")
stats_file.write(f"The standard deviation of the exact centralities is: {std_dev_exact_centralities}\n\n")
stats_file.write(f"Results of the approximation algorithm with epsilon = 0.1\n")
stats_file.write(f"The mean of the approximated centralities is: {mean_approximated_centralities_1}\n")
stats_file.write(f"The standard deviation of the approximated centralities is: {std_dev_approximated_centralities_1}\n")
stats_file.write(f"The mean of the errors is: {mean_errors_1}\n")
stats_file.write(f"The standard deviation of the errors is: {std_dev_errors_1}\n\n")
    
# Close opend file
f.close()
    

Let's try to run the approximation algorithm again with epsilon = 0.05

In [ ]:
#let's compute the value k of the number of iteration we have to do with epsilon = 0.05
k = int(math.log(G.number_of_nodes(),2)/0.0025)

start_time = time.time()
approximated_closeness_centrality_2 = ApproximateClosenessCentrality(G,k)
end_time = time.time()

#let's sort the results based on the value
approximated_closeness_centrality_2 = {k: v for k, v in sorted(approximated_closeness_centrality_2.items(),reverse=True , key=lambda item: item[1])}


#let's print the results
#print(approximated_closeness_centrality_2)
print("the approximated closeness centralities with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

#Let's store the results
f = open('results/results_approximated_closeness_centrality_epsilon_0_0_5.txt', "w", encoding="utf8")
for key, value in approximated_closeness_centrality_2.items():
    f.write('%s:%s\n' %(key, value))
    
# Close opend file
f.close()

Now let's see the result by calculating the statistic as we did previously

In [ ]:
mean_exact_centralities = 0
mean_approximated_centralities_2 = 0
mean_errors_2 = 0
std_dev_exact_centralities = 0
std_dev_approximated_centralities_2 = 0
std_dev_errors_2 = 0
G_size = G.number_of_nodes()

#First we compute the means
#let's start by summing
for n in G:
    mean_exact_centralities += exact_closeness_centrality[n]
    mean_approximated_centralities_2 += approximated_closeness_centrality_2[n]
mean_errors_2 = mean_approximated_centralities_2 - mean_exact_centralities

#now let's divide by the number of nodes to compute the final values
mean_exact_centralities = mean_exact_centralities/G_size
mean_approximated_centralities_2 = mean_approximated_centralities_2/G_size
mean_errors_2 = mean_errors_2/G_size

#Now we compute the standard deviations
#Let's start by computing the summation
for n in G:
    tmp = exact_closeness_centrality[n] - mean_exact_centralities
    std_dev_exact_centralities += tmp*tmp
    tmp = approximated_closeness_centrality_2[n] - mean_approximated_centralities_2
    std_dev_approximated_centralities_2 += tmp*tmp
    tmp = approximated_closeness_centrality_2[n] - exact_closeness_centrality[n] - mean_errors_2
    std_dev_errors_2 += tmp*tmp

#Let's now compute the final value dividing by the size and computing the square root of the variance
std_dev_exact_centralities = (std_dev_exact_centralities/G_size)**0.5
std_dev_approximated_centralities_2 = (std_dev_approximated_centralities_2/G_size)**0.5
std_dev_errors_2 = (std_dev_errors_2/G_size)**0.5

#Let's print the results regarding the approximated centralities and their errors
print("Results of the approximation algorithm with epsilon = 0.05")
print("The mean of the approximated centralities is:", mean_approximated_centralities_2)
print("The standard deviation of the approximated centralities is:", std_dev_approximated_centralities_2)
print("The mean of the errors is:", mean_errors_2)
print("The standard deviation of the errors is:", std_dev_errors_2)

#Let's print the top 10 computed with approximated algorithm
position=1
for key, value in approximated_closeness_centrality_2.items():
    key_node = G.nodes()[key]
    print(f'position number {position}:')
    print(f"   artist: {key_node['artist']}\n   followers: {key_node['followers']}\n   popularity: {key_node['popularity']}\n   centrality: {value}")
    position +=1
    if position > 10:
        break
        
#Let's store the results
stats_file.write(f"Results of the approximation algorithm with epsilon = 0.05\n")
stats_file.write(f"The mean of the approximated centralities is: {mean_approximated_centralities_2}\n")
stats_file.write(f"The standard deviation of the approximated centralities is: {std_dev_approximated_centralities_2}\n")
stats_file.write(f"The mean of the errors is: {mean_errors_2}\n")
stats_file.write(f"The standard deviation of the errors is: {std_dev_errors_2}\n\n")

# ToDo generate random graph and run algorithms on said graph, do it j times and compute statistics (also p score with our graph)

Now that we computed our graph's centralities and the average we need to know if it's a property or if it's a normal value compared to graphs of proportional size.

To do so we randomly generate graphs with as much nodes and edges as our graph, making sure that the graph that we obtain is connected, then we compute the stats for these graphs and compare them with the stats previously computed

Let's start by generating 10 random graphs without taking in consideration the degree of the nodes and for each graph let's sort the centralities and store them so that we can confront the results and let's also compute the average

In [ ]:
#let's compute the probability of an edge as number of edges over possible number of edges
probability = G.number_of_edges()/(G.number_of_nodes()*(G.number_of_nodes()-1)/2)

for i in range(10):
    #Let's now inizialize the random graph, as seed we use a pseudo-random number
    random_G = nx.fast_gnp_random_graph(G.number_of_nodes(), p = probability, seed = rnd.randrange(10000))
    
    #If the first graph we generated is not connected we keep generating graphs until we find a connected graph
    while not nx.is_connected(random_G):
        random_G = nx.fast_gnp_random_graph(G.number_of_nodes(), p = probability, seed = rnd.randrange(10000))
    
    #Now that we generated the graph we need to compute the approximated centraliteis
    k = int(math.log(random_G.number_of_nodes(),2)/0.01)
    
    start_time = time.time()
    approximated_closeness_centrality_random = ApproximateClosenessCentrality(random_G,k)
    end_time = time.time()
    
    #let's sort the results based on the value
    approximated_closeness_centrality_random = {k: v for k, v in sorted(approximated_closeness_centrality_random.items(),reverse=True , key=lambda item: item[1])}

    #let's print the results
    #print(approximated_closeness_centrality_random)
    print(f"the approximated closeness centralities of the {i+1} random graph with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

    #Let's store the results and compute the sum of all the centralities that we will use later to compute the mean
    mean_random_centralities = 0
    f = open(f'results/results_approximated_closeness_centrality_random_{i+1}.txt', "w", encoding="utf8")
    for key, value in approximated_closeness_centrality_random.items():
        f.write('%s:%s\n' %(key, value))
        mean_random_centralities += value
    
    #now let's divide by the number of nodes to compute the mean
    mean_random_centralities = mean_random_centralities/random_G.number_of_nodes()
    
    #And finally Let's store the average in the stats file
    stats_file.write(f"The mean of the approximated centralities of the {i} random graph is: {mean_random_centralities}\n")

Now let's do the same thing but let's generate 10 random graphs taking in consideration the degree of the nodes

If the returned graph is not connected we randomly add enough edges to make the graph connected

In [12]:
expected_degrees = []
for n in G:
    expected_degrees.append(G.degree[n])

for i in range(10):
    #Let's now inizialize the random graph, as seed we use a pseudo-random number
    random_degree_G = nx.expected_degree_graph(expected_degrees, seed=rnd.randrange(10000), selfloops=False)
    
    #If the graph we generated is not connected we randomly add enough edges to make it connected
    if not nx.is_connected(random_degree_G):
        #we initialize a list where we will put 2 node for each connected component,
        #one for incoming edge on connected commponent and the second for the outgoing one
        random_nodes = []    
        for component in nx.connected_components(random_degree_G):
            random_nodes.append(rnd.choice(list(component)))
            random_nodes.append(rnd.choice(list(component)))

        #now we create a path between the nodes in the list
        for j in range(int(len(random_nodes)/2)-1):
            random_degree_G.add_edge(random_nodes[2*j+1], random_nodes[2*j+2])
    
    #Now that we generated the graph we need to compute the approximated centraliteis
    k = int(math.log(random_degree_G.number_of_nodes(),2)/0.01)
    
    start_time = time.time()
    approximated_closeness_centrality_random_degree = ApproximateClosenessCentrality(random_degree_G,k)
    end_time = time.time()
    
    #let's sort the results based on the value
    approximated_closeness_centrality_random_degree = {k: v for k, v in sorted(approximated_closeness_centrality_random_degree.items(),reverse=True , key=lambda item: item[1])}

    #let's print the results
    #print(approximated_closeness_centrality_random_degree)
    print(f"the approximated closeness centralities of the {i+1} random graph that takes in consideration the degree of the nodes with %s iterations have been computed in %s seconds" %(k, end_time - start_time))

    #Let's store the results and compute the sum of all the centralities that we will use later to compute the mean
    mean_random_degree_centralities = 0
    f = open(f'results/results_approximated_closeness_centrality_random_wiht_same_degree_{i+1}.txt', "w", encoding="utf8")
    for key, value in approximated_closeness_centrality_random_degree.items():
        f.write('%s:%s\n' %(key, value))
        mean_random_degree_centralities += value
        
    #now let's divide by the number of nodes to compute the mean
    mean_random_degree_centralities = mean_random_degree_centralities/random_degree_G.number_of_nodes()
    
    #And finally Let's store the average in the stats file
    stats_file.write(f"The mean of the approximated centralities of the {i} random graph that takes in consideration the degree of the nodes is: {mean_random_degree_centralities}\n")

# Close opend file
stats_file.close()

the approximated closeness centralities of the 1 random graph that takes in consideration the degree of the nodes with 737 iterations have been computed in 0.3220658302307129 seconds
the approximated closeness centralities of the 2 random graph that takes in consideration the degree of the nodes with 737 iterations have been computed in 0.29790401458740234 seconds
the approximated closeness centralities of the 3 random graph that takes in consideration the degree of the nodes with 737 iterations have been computed in 0.40824031829833984 seconds
the approximated closeness centralities of the 4 random graph that takes in consideration the degree of the nodes with 737 iterations have been computed in 0.5100886821746826 seconds
the approximated closeness centralities of the 5 random graph that takes in consideration the degree of the nodes with 737 iterations have been computed in 0.38060593605041504 seconds
the approximated closeness centralities of the 6 random graph that takes in consid